Следующее задание такое.

Нужно выбрать для себя новостной сайт.
Заведите себе гуглдок и запишите туда свои сайты -- чтобы не повторяться.

В репозитории должен лежать готовый код, который умеет делать всё, что я перечислю ниже, а также в файле Readme.md в папке с проектом должна находиться ссылка на скачанные и обработанные файлы вашего новостного сайта, лежащие архивом на Яндекс.Диске, Google Drive, Облаке@Мэйл.ру или на чём угодно подобном.
При этом 
1. Период, за который должны быть тексты: 1 июля 2015 -- 31 декабря 2018 года.
2. Общий объём скачанных текстов должен быть не меньше 5 млн. слов (можно больше). 
3. В именах файлов не должно быть не-ascii символов.
4. Структура каталогов, в которых хранятся файлы, должна быть следующей: корень/год/месяц/файлы с материалами за этот месяц
5. В корне также должна лежать csv-таблица с полями, которые я перечислю ниже. 

Соответственно, сама программа (в одном или в нескольких файлах) должна уметь:
1. написать программу, которая умеет автоматически обращаться к сайту газеты и скачивать оттуда html-страницы, при этом программа должна уметь находить ссылки на другие страницы этого сайта и скачивать их тоже.
При этом программа не должна заходить на одни и те же страницы несколько раз.
2. с помощью lxml (или другого парсера xml) извлекать со страниц информацию (если она доступна) для метатаблицы и сам текст.
Метатаблица должна иметь следующие поля: 
path;author;date;source;title;url;wordcount
path -- это путь к файлу. 
source -- это название новостного сайта. 
wordcount -- число слов в тексте.
Остальное, думаю, понятно (если вдруг непонятно, пишите).
3. раскладывать скачанные тексты по папкам (в соответствии с теми правилами, которые изложены выше).
4. вызывать mystem и делать морфологическую разметку текста (таким образом, чтобы каждому слову была присвоена информация о лемме и грамматическая информация с учётом контекстно снятой омонимии).
Размеченные майстемом файлы должны лежать в отдельной папке. То есть структура каталога должна быть такая:

корень
|
|___ plain text
|      |___год
|            |___месяц
|___ размеченные майстемом тексты 
       |___год
             |___месяц
Файлы, размеченные майстемом, записывать в метатаблицу не надо.

Сделать всё нужно до 23:59 28 февраля 2019 года.

Всего доброго,
Борис Орехов

In [191]:
import requests
from bs4 import BeautifulSoup as bs
from datetime import datetime, timedelta
import time
import re
import html5lib
import urllib.request
import os
import pandas as pd
from pymystem3 import Mystem
m = Mystem()

In [35]:
site = 'https://hi-news.ru/'
sitemap = 'https://hi-news.ru/sitemap.xml'
end_date = '2015-07'

In [6]:
def parse_sitemap(sitemap, end_date):
    # path;author;date;source;title;url;wordcount
    # path: re.sub('\-', '/', date[:-3])
    # 
    
    end_date = int(''.join(re.findall('\d+', end_date)))
    print (end_date)
    resp = requests.get(sitemap)
    links = str(bs(resp.text, 'lxml'))
    links = re.findall('https[^<]+\d\.xml', links)
    articles = []
    count_links = 0
    for link in links:
        date = int(''.join(re.findall('\d+', link)))
        if date >= end_date:
            count_links += 1
            arts = requests.get(link)
            arts = str(bs(arts.text, 'lxml'))
            arts = re.findall('https[^<]+\.html', arts)
            articles += arts
    print ('{} xml links parsed'.format(count_links))
    print ('{} articles collected'.format(len(articles)))
    return articles    

In [7]:
links = parse_sitemap(sitemap, '2015-07')

201507
48 xml links parsed
10532 articles collected


In [286]:

def pos(text):
    pos_text = ''
    for a in m.analyze(text):
        
        try:
            text = a['text']
            lemma = a['analysis'][0]['lex']
            gr = a['analysis'][0]['gr']
            if a['text'].lower() == lemma:
                lemma = ''
            if '|' in gr:
                gr = ''

            output = str([i for i in [lemma, gr] if len(i) > 0])
            if len(output) == 0:
                output = ''
            pos_text += text + output
        except KeyError:
            pos_text += a['text']
        except IndexError:
            pos_text += a['text']
    return pos_text.strip('\n')

def parse_articles(list_of_articles):
    df = pd.DataFrame(columns=['path','date','source','title','url','wordcount'])
    path = os.path.abspath('.') + '/news/'
    for article in list_of_articles:
        art_text = requests.get(article).text
        
        #art_text = requests.get(links[0]).text
        
        date = re.findall('\"datePublished": \"(\d{4}-\d\d-\d\d)', art_text)[0]
        #print (date)
        page = requests.get(article)
        art_text_clean = bs(' '.join([re.sub('[ ]{2,}', '', p.get_text()) for p in bs(page.text, 'lxml').\
                     find_all('p')]), 'lxml').get_text().replace('\xa0', ' ').replace('\n', '')

        #art_text_clean = re.findall('<div class="text">([^{]+){', art_text)#[0]

        title = re.findall('<title>([^<]+)</title>', art_text)[0][:-13]
        file_title = re.sub('[^a-zа-яё]+', '_', title.lower()).strip('_')
        #print (title)
        author = re.findall('class=\"author[^>]+>([^<]+)<', art_text)[0]
        #print (author)
        #print (art_text_clean)
        dir_path = path + 'plain_text' + '/' +  re.sub('-', '/', date)[:-3]
        file_path =  dir_path + '/' + file_title + '.txt'
        mystem_path = path + 'pos_tag' + '/' +  re.sub('-', '/', date)[:-3]
        #print (file_path)
        
        try:
            os.makedirs(dir_path, exist_ok=True)
        except IsADirectoryError:
            pass
        except FileExistsError:
            pass
        
        f = open(file_path, 'w')
        f.write(art_text_clean)
        f.close()
        wordcount = len(re.sub('[^a-zа-яё ]', '', art_text_clean.lower()).split())
        df = df.append({'path':file_path,'date':date,'source':site,'title':title,
                        'url':article,'wordcount':wordcount}, ignore_index=True)
        
        pos_text = pos(art_text_clean)
        try:
            os.makedirs(mystem_path, exist_ok=True)
        except IsADirectoryError:
            pass
        except FileExistsError:
            pass
        
        f = open(mystem_path + '/' + file_title + '.txt', 'w')
        f.write(pos_text)
        f.close()
        
    return df
        

In [ ]:
%time df = parse_articles(links)
df.to_csv('hi_news_articles.csv')

In [290]:
len(links)

10532